In [1]:
#Mengimport semua library yang dibutuhkan 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, r2_score
import pickle

In [3]:
air_quality_dataset = pd.read_csv('air_quality_health_impact_data.csv', on_bad_lines="skip")
print(air_quality_dataset.head())

   RecordID         AQI        PM10       PM2_5         NO2        SO2  \
0         1  187.270059  295.853039   13.038560    6.639263  66.161150   
1         2  475.357153  246.254703    9.984497   16.318326  90.499523   
2         3  365.996971   84.443191   23.111340   96.317811  17.875850   
3         4  299.329242   21.020609   14.273403   81.234403  48.323616   
4         5   78.009320   16.987667  152.111623  121.235461  90.866167   

           O3  Temperature   Humidity  WindSpeed  RespiratoryCases  \
0   54.624280     5.150335  84.424344   6.137755                 7   
1  169.621728     1.543378  46.851415   4.521422                10   
2    9.006794     1.169483  17.806977  11.157384                13   
3   93.161033    21.925276  99.473373  15.302500                 8   
4  241.795138     9.217517  24.906837  14.534733                 9   

   CardiovascularCases  HospitalAdmissions  HealthImpactScore  \
0                    5                   1          97.244041   
1   

In [4]:
air_quality_dataset.isna().sum()

RecordID               0
AQI                    0
PM10                   0
PM2_5                  0
NO2                    0
SO2                    0
O3                     0
Temperature            0
Humidity               0
WindSpeed              0
RespiratoryCases       0
CardiovascularCases    0
HospitalAdmissions     0
HealthImpactScore      0
HealthImpactClass      0
dtype: int64

In [6]:
def count_outliers (air_quality_dataset):
    col_numerik = air_quality_dataset.select_dtypes(include=[np.number]).columns
    outliers = {}
    for col in col_numerik :
        limit_atas = air_quality_dataset[col].quantile(0.99)
        outliers[col] = (air_quality_dataset[col] > limit_atas).sum()
        return outliers
    
outlier_air_quality_dataset = count_outliers(air_quality_dataset.drop(columns = ['RecordID']))

def remove_outliers(air_quality_dataset):
    col_numerik = air_quality_dataset.select_dtypes(include=[np.number]).columns
    for col in col_numerik:
        limit_atas = air_quality_dataset[col].quantile(0.99)
        air_quality_dataset = air_quality_dataset[air_quality_dataset[col] <= limit_atas]
    return air_quality_dataset

cleaned_air_dataset = remove_outliers(air_quality_dataset) 

print("\nData Train sebelum di hapus outliers",air_quality_dataset.shape)
print("\nData Train setelah di hapus outliers",cleaned_air_dataset.shape)


Data Train sebelum di hapus outliers (5811, 15)

Data Train setelah di hapus outliers (5123, 15)


In [9]:
x = cleaned_air_dataset.drop(['HealthImpactScore','HealthImpactClass'], axis=1)
x

,RecordID,AQI,PM10,PM2_5,NO2,SO2,O3,Temperature,Humidity,WindSpeed,RespiratoryCases,CardiovascularCases,HospitalAdmissions
0,1,187.270059,295.853039,13.038560,6.639263,66.161150,54.624280,5.150335,84.424344,6.137755,7,5,1
1,2,475.357153,246.254703,9.984497,16.318326,90.499523,169.621728,1.543378,46.851415,4.521422,10,2,0
2,3,365.996971,84.443191,23.111340,96.317811,17.875850,9.006794,1.169483,17.806977,11.157384,13,3,0
4,5,78.009320,16.987667,152.111623,121.235461,90.866167,241.795138,9.217517,24.906837,14.534733,9,0,1
5,6,77.997260,36.113445,97.113240,87.769562,32.261206,136.999714,-1.441781,32.635904,4.675127,13,5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5741,5742,284.748629,243.671862,38.236608,166.237591,81.559944,79.553510,12.205770,26.013018,4.270559,7,4,4
5744,5745,81.808102,169.443834,145.418546,60.850657,1.929838,37.976945,-2.453523,72.323302,8.561057,13,4,1
5746,5747,337.146787,246.493979,130.210223,92.594733,58.776887,200.775019,11.579942,91.232370,6.218481,8,8,2
5747,5748,489.977724,36.686616,43.492069,119.508875,96.194297,112.458508,1.254843,44.262755,6.106693,9,2,1


In [10]:
scaler = StandardScaler()
x_standard = scaler.fit_transform(x)
x_standard

array([[-1.73963865, -0.40687929,  1.74615026, ..., -0.96775491,
         0.01505971, -0.72115547],
       [-1.73903148,  1.61018516,  1.1622499 , ...,  0.0236085 ,
        -1.37088415, -1.46421249],
       [-1.7384243 ,  0.84449124, -0.74268902, ...,  1.01497191,
        -0.90890287, -1.46421249],
       ...,
       [ 1.74919893,  0.64249441,  1.1650668 , ..., -0.63730044,
         1.40100357,  0.02190154],
       [ 1.74980611,  1.71255223, -1.3049071 , ..., -0.30684597,
        -1.37088415, -0.72115547],
       [ 1.75041328,  1.73231519,  1.26021653, ..., -1.62866385,
        -0.44692158,  0.02190154]])

In [17]:
y = cleaned_air_dataset['HealthImpactClass']
y

0       0.0
1       0.0
2       0.0
4       0.0
5       1.0
       ... 
5741    2.0
5744    1.0
5746    1.0
5747    3.0
5748    1.0
Name: HealthImpactClass, Length: 5123, dtype: float64

In [14]:
x_train,x_test,y_train,y_test=train_test_split(x_standard, y, test_size=0.2, random_state=42)

In [18]:
model_classifier_2 = RandomForestClassifier()
model_classifier_2.fit(x_train,y_train)

y_pred = model_classifier_2.predict(x_test)
accuracy_test = accuracy_score(y_test, y_pred)
print(accuracy_test)

ValueError: Unknown label type: continuous. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.